# Project III: Classical Non-linear Models and Police use of force
This notebook uses the Police Public Contact Survey (PPCS) dataset: `ppcs_cc.csv`.

In [17]:
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import probit as probit
import logit as logit
import estimation as est
from scipy.stats import norm
from scipy.stats import t

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load the data 

In [2]:
# Load the dataset
dat = pd.read_csv('ppcs_cc.csv')

# Inspect distribution of the target variable
print("\nDistribution of 'anyuseofforce_coded':")
print(dat['anyuseofforce_coded'].value_counts(normalize=True))

# Inspect value counts for categorical variables
categorical_vars = ["sblack", "shisp", "swhite", "sother", "smale", "omajblack", 
                    "omajhisp", "omajwhite", "omajother", "osplit", "inctype_lin", "sbehavior"]

for var in categorical_vars:
    print(f"\nValue Counts for {var}:")
    print(dat[var].value_counts())


Distribution of 'anyuseofforce_coded':
anyuseofforce_coded
0    0.994999
1    0.005001
Name: proportion, dtype: float64

Value Counts for sblack:
sblack
0    3379
1     420
Name: count, dtype: int64

Value Counts for shisp:
shisp
0    3413
1     386
Name: count, dtype: int64

Value Counts for swhite:
swhite
1    2808
0     991
Name: count, dtype: int64

Value Counts for sother:
sother
0    3614
1     185
Name: count, dtype: int64

Value Counts for smale:
smale
1    2012
0    1787
Name: count, dtype: int64

Value Counts for omajblack:
omajblack
0    3568
1     231
Name: count, dtype: int64

Value Counts for omajhisp:
omajhisp
0    3708
1      91
Name: count, dtype: int64

Value Counts for omajwhite:
omajwhite
1    3433
0     366
Name: count, dtype: int64

Value Counts for omajother:
omajother
0    3755
1      44
Name: count, dtype: int64

Value Counts for osplit:
osplit
0    3799
Name: count, dtype: int64

Value Counts for inctype_lin:
inctype_lin
2    3641
1     158
Name: count, dtype

In [3]:
# Declare labels    
y_lab = 'anyuseofforce_coded'
#x_lab = ['const', 'sblack', 'shisp', 'sother']

#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'sage']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior','sagesq']

dat['sage'] = dat['sage'] / 10
dat['sagesq'] = dat.sage * dat.sage 

#['const', 'sblack', 'shisp', 'sother', 'smale', 'sage', 'sempl', 'sincome', 'spop', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior', 'year']  # Dropped 'osplit' and year // Multikollinearitet mellem hhv. "omajblack", "omajhisp" "omajwhite" og "omajother" +  "sblack", "shisp", "swhite", "sother"

## Declare labels, her er sother og omajother taget ud, men I kan bare rette til white
#y_lab = 'anyuseofforce_coded'
#x_lab = ["const", "sblack", "shisp", "swhite",  # Drop 'sother' giver Multikollinearitet  sammen med "sblack", "shisp", "swhite"
#         "smale", "sage", "sempl", "sincome", 
#         "spop", "daytime", "inctype_lin", 
#         "omajblack", "omajhisp", "omajwhite",  # Drop 'omajother' giver Multikollinearitet sammen med "omajblack", "omajhisp", "omajwhite"
#         "sbehavior"] # dropped year and osplit (indeholder kun 0?)

# create extra variables 
N = dat.shape[0]
dat['const'] = np.ones((N,))

# Rebuild the dataset without 'osplit'
dat = dat[[y_lab] + x_lab].copy()

# Check for missing data
assert dat.notnull().all(axis=1).all(), 'Missing values detected. Clean your data!'

dat.tail(5)

,anyuseofforce_coded,const,sblack,shisp,sother,smale,sempl,sincome,spop,sage,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
3794,0,1.0,0,0,0,0,1,3,1,7.2,0,1,0,0,0,1,51.84
3795,0,1.0,0,0,0,0,0,2,1,7.1,1,2,0,0,0,0,50.41
3796,0,1.0,0,0,0,0,0,1,1,7.6,1,2,0,0,0,0,57.76
3797,0,1.0,0,0,0,0,0,3,4,7.9,1,2,0,0,0,0,62.41
3798,0,1.0,0,0,0,0,0,2,1,7.5,1,2,0,0,0,0,56.25


In [4]:
# Extract y and X
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

## Estimate using Probit

In [5]:
# Initialize starting values
theta0 = probit.starting_values(y, x)

# Display starting values
print("Starting values for theta:", theta0)

probit_results = est.estimate(probit.q, theta0, y, x, cov_type='Sandwich')

Starting values for theta: [ 0.13618175  0.00493452  0.02218758 -0.00094007  0.01082135 -0.0155645
  0.00220925  0.01026053  0.00248039 -0.00447688 -0.07037371 -0.01365712
 -0.00313071 -0.01591562  0.08840108 -0.00072741]
Optimization terminated successfully.
         Current function value: 0.022253
         Iterations: 165
         Function evaluations: 3060
         Gradient evaluations: 180


In [6]:
probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
probit_tab

Optimizer succeded after 165 iter. (3060 func. evals.). Final criterion:  0.02225.
Probit, y = anyuseofforce_coded


,theta,se,t
const,-2.6163,0.6957,-3.7608
sblack,0.2042,0.2990,0.6830
shisp,0.4162,0.2385,1.7453
sother,0.1051,0.4415,0.2380
smale,0.5619,0.2102,2.6734
sempl,-0.5001,0.2206,-2.2673
sincome,0.1022,0.1244,0.8215
spop,0.1988,0.0747,2.6626
sage,0.4862,0.3720,1.3071
daytime,-0.1416,0.1980,-0.7151


## Estimate using Logit

In [7]:
logit_results = est.estimate(logit.q, theta0, y, x, cov_type='Sandwich')

Optimization terminated successfully.
         Current function value: 0.022389
         Iterations: 201
         Function evaluations: 3604
         Gradient evaluations: 212


In [8]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeded after 201 iter. (3604 func. evals.). Final criterion:  0.02239.
Logit, y = anyuseofforce_coded


,theta,se,t
const,-5.6082,1.9379,-2.8939
sblack,0.4375,0.8072,0.5420
shisp,0.9407,0.6116,1.5380
sother,-0.0905,1.2067,-0.0750
smale,1.1538,0.5734,2.0122
sempl,-1.1621,0.6153,-1.8887
sincome,0.2021,0.3421,0.5909
spop,0.4811,0.1889,2.5476
sage,1.3022,1.1781,1.1054
daytime,-0.3877,0.5327,-0.7278


## Average partial effects

### Probit

In [9]:
# Estimating the average partial effects 
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother'] 
probit.properties(x, probit_results['theta'],probit_results['cov'],print_out = True,se=True,indices=indices, labels = labels)

,Estimate,SE,t-value,p-value
sblack,0.003,0.004,0.589,0.556
shispanic,0.006,0.005,1.333,0.183
sother,0.001,0.006,0.217,0.828


### Logit

In [10]:
# Estimating the average partial effects 
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother']  
logit.properties(x, logit_results['theta'],logit_results['cov'],print_out = True,se=True,indices=indices, labels = labels)

,Estimate,SE,t-value,p-value
sblack,0.002,0.005,0.472,0.637
shispanic,0.005,0.005,1.181,0.238
sother,-0.000,0.005,-0.078,0.938


In [84]:
# Let us make a vector of the values we want to investigate
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sage', 'sempl', 'sincome', 'spop', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0, 1, 4.1, 0, 2.16, 1.36,0,1,0,0,0,0,19.42]).reshape(1, -1)
pd.DataFrame(x_me, columns=x_lab, index=['x_me'])


,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me,1.0,0.0,0.0,0.0,1.0,4.1,0.0,2.16,1.36,0.0,1.0,0.0,0.0,0.0,0.0,19.42


In [91]:
# We will look at the same values as previously, but we want to look at the difference for foreign = 0 and foreign = 1.
k = 3
x_me2 = x_me.copy()
x_me2[:, k] = 1  # Keep everythin the same, but change foreign to 1 for all obs. 
pd.DataFrame(x_me2, columns=x_lab, index=['x_me2'])

,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me2,1.0,0.0,0.0,1.0,1.0,4.1,0.0,2.16,1.36,0.0,1.0,0.0,0.0,0.0,0.0,19.42


In [92]:
b_pr = probit_tab.theta.values
b_lg = logit_tab.theta.values

In [93]:

# evaluate G at the two different "x beta"s
me_black_pr = norm.cdf(x_me2 @ b_pr)-norm.cdf(x_me @ b_pr)

me_black_lg = ((np.exp(np.dot(x_me2, b_lg))) / ((1 + np.exp(np.dot(x_me2, b_lg)))**2))-((np.exp(np.dot(x_me, b_lg))) / ((1 + np.exp(np.dot(x_me, b_lg)))**2))

# print results 
pd.DataFrame([me_black_pr[0], # assuming me_foregin_pr is an array, otherwise remove "[0]"
           me_black_lg[0]],
          index=['Probit', 'Logit'], columns=[f'Marg. Eff.: {x_lab[k]}']).round(6)

,Marg. Eff.: sother
Probit,0.0
Logit,-0.0


In [94]:
bb = np.outer(b_pr,b_pr)
xx = np.outer(x_me, x_me)
I_K = np.eye(K)
gx0 = norm.pdf(x_me@b_pr)
gx2 = norm.pdf(x_me2@b_pr)

grad_c_pr = gx0*(np.eye(K) - bb @ xx)
grad_d_pr = gx2*x_me2 - gx0*x_me

In [95]:
def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_c_pr = get_se(grad_c_pr, probit_results['cov'])
se_d_pr = get_se(grad_d_pr, probit_results['cov'])

p_values = 2 * t.sf(np.abs(me_black_pr[0]/se_d_pr), df=x.shape[0] - x.shape[1]).round(4)

In [96]:
me_dict = {'Marginal Effect': np.vstack([me_black_pr])[:,0],
           's.e.':            np.vstack([se_d_pr])[:,0]}
tab = pd.DataFrame(me_dict,index=['black'])
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

,Marginal Effect,s.e.,t
Var,,,
black,0.0,0.0,0.100232


In [76]:
p_values = 2 * t.sf(np.abs(me_black_pr[0]/se_d_pr), df=x.shape[0] - x.shape[1]).round(4)
print(p_values)

[0.8832]
